* 오차역전파법 Backpropagation : 오차를 역으로 전파 하는 방법 

# 5.1 계산 그래프 

계산 과정을 그래프로 나타낸 것 : Node(노드)와 Edge(에지) 로 표현

## 5.1.1 계산 그래프로 풀다 

[사과 한개] --100원--> [*2 개]  --200원-->  [**1.1 수수료] --220원-->  결제 

1. 계산 그래프를 구성한다. 
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. 

* forward propagation : 계산을 왼쪽에서 오른쪽으로 진행
* backward propagation : 계산을 오른쪽에서 왼쪽으로 진행 

## 5.1.2 국소적 계산 
국소적 : 자신과 직접 관계된 작은 범위 
<img width=50% src ='calculateGraph.png' ></img>


## 5.1.3 왜 계산 그래프?
사과 값이 조금 올랐다. 지불금액이 얼마나 증가 하는가? <br>
--> 역전파를 이용해 구할 수 있다. 

> 사과 가격에 대한 지불금액의 미분 <br>
사과 가격이 1원 오르면 최종 금액은 2.2원 오르게 된다. <br>
오른쪽에서 왼쪽으로 1 -> 1.1 > 2.2 순으로 미분 값을 전달

# 5.2 Chain rule (연쇄법칙)

## 5.2.1 계산 그래프의 역전파 

신호 E 에 순전파 때의 계산의 미분을 곱한 후 다음 노드로 전달한다. <br>
f(x) 가 $x^2$ 라면 미분한 값은 2x 가 된다. 

## 5.2.2 연쇄법칙이란?
* 합성 함수 : 여러 함수로 구성된 함수 
    * 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타냄 <br>

$z = (x+y)^2$   <br>
        =><br>
        $z = t^2$<br>
        $t = (x+y)$

각각에 대해서 편미분 하면 x 에 대한 z 의 미분 값은 = 2(x+y) 가 된다. 

# 5.3 역전파

## 5.3.1 덧셈 노드의 역전파 

$z=x+y$ 를 미분 

덧셈 노드의 역전파는 1을 곱하기만 할뿐 입력되는 값을 그대로 다음 노드로 보냄

입력신호를 다음 노드로 출력할 뿐 그대로 다음노드로 전달함 

## 5.3.2 곱셈 노드의 역전파 

$z=xy$ 를 미분

곱셈 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값' 을 곱해서 하류에 보냄

10, 5 x -> 50 이 됐으면 곱셈의 역전파에서는 <br>
1.3 x 5 = 6.5 다른 하나는 1.3 x 10 = 13 이 되어 <br>
6.5 는 10 이 온 방향으로 <br>
13 은 5 가 온 방향으로 보내짐 

# 5.4 단순 계층 구현

